#  Creating the Correlation vs GPS Time Dataframe

**Alex B. Nielsen<sup>1</sup>, Felicia Frederiksson<sup>2</sup>, Paolo Marcoccia<sup>1</sup> and Germano Nardini<sup>1</sup>**

<sub>1. University of Stavanger, Institutt for Matematikk og Fysikk, Kjølv Egelands hus, 5.etg, E-blokk, 4021 Stavanger, Norway </sub> <br>
<sub>2. University of Uppsala, Department of Physics and Astronomy,Ångströmlaboratoriet, Lägerhyddsvägen 1, 751 20 Uppsala, Sweden</sub> 

We encourage use of these data in derivative works. If you use the material provided here, please cite [our paper.]()

In this notebook, we will learn how to generate the [CorrVsTime.csv](https://github.com/GravWaves-IMF/Correlation-Method-first-2019-/blob/master/Code/GW151012Final/GW151012data.py) dataframe containing the Correlation vs GPS Time for the analyzed events.
The pipeline, in particular, will be run for the event _GW151012_ but it may easily be applied to all the other events.
In order to run this pipeline, however, you need the _residuals.hdf_ file generated by running the [CreateResiduals.ipynb](https://github.com/gwastro/gw150914_investigation/blob/master/CreateResiduals.ipynb) notebook.
Let's start my moving into the directory of the event we wish to analyze :

In [1]:
cd GW151012Final/

/home/kuzaibeir/Documenti/IPyNb/GW151012Final


In each event directory, there will be a [init_module.py](https://github.com/GravWaves-IMF/Correlation-Method-first-2019-/blob/master/Code/init_module.py) and a [GW*event_name*.py](https://github.com/GravWaves-IMF/Correlation-Method-first-2019-/blob/master/Code/GW151012Final/GW151012data.py), the first one once launched will automatically load all the modules needed to run the pipeline, the latter instead will set some local variables around the choosed event.
Let's launch them both :

In [2]:
%run init_module.py
%run GW151012data.py

/home/kuzaibeir/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


segment length: 16


Let's also inizialize some void dictionary :


In [3]:
data, psd_data, ts, ts3, waveforms, psds = {},{},{},{},{},{}

We will analyze the correlations between the two detectors in a _200 ms_ time interval centered around the _LIGO_ claimed coalescence time.
Furthermore, we will estimate the correlations each _0.1 ms_, hence we will estimate _2000_ correlations in total during the following pipeline.
Let's generate some auxiliary vectors in order to do so :

In [4]:
rangenum = range(2000)
timespan = np.linspace(0.,0.2,2000)
void = np.zeros(2000)

Now, we need to load both the Strain Data of the detectors at the time of event and the Residual data obtained by running the [CreateResiduals.ipynb](https://github.com/gwastro/gw150914_investigation/blob/master/CreateResiduals.ipynb) notebook.
The Strain Data of the events may be downloaded at the [GWOSC](https://www.gw-openscience.org/catalog/GWTC-1-confident/) website, however depending on how old your strain data is, it may be saved with both the header name _LOSC_ or _GWOSC_.
In our case, the data of _GW150914_ was saved with the _LOSC_ header name, while the data of the other events was saved with the _GWOSC_ header, for avoiding failing in the loading of the strain data, one should check carefully and change the <em>hd_nm</em> variable in the [GW*event_name*.py](https://github.com/GravWaves-IMF/Correlation-Method-first-2019-/blob/master/Code/GW151012Final/GW151012data.py) script according to the data.
Furthermore, in the [res.py](https://github.com/GravWaves-IMF/Correlation-Method-first-2019-/blob/master/Code/res.py) file, there would be a <em>res.get_LSC_Full_strain()</em> that was built to load _LOSC_ header file, and a <em>res.get_GWSC_Full_strain()</em> that was built for _GWOSC_ header file.
In our case, for _GW151012_ will be :

In [5]:
strain = res.get_GWSC_Full_strain(hd_nm,psd_start_time-pad_data, psd_end_time + pad_data)
resstrain = res.get_Full_residual_strain()

Let's also inizialize an auxiliary dataframe, that will be used to save the correlation data :

In [6]:
df = pd.DataFrame({'GPSTime' : basetime + timespan, 
     'Timeshift' : timespan,
     'SigCorr' : void,
     'ResCorr' : void,
     'DiffCorr' : void})

Now let's cut down the data around the event, and let's whiten that in the frequency band defined by <em>f_low</em>, <em>f_high</em>, the values of the two previous variable are stated for each event in the [GW*event_name*.py](https://github.com/GravWaves-IMF/Correlation-Method-first-2019-/blob/master/Code/GW151012Final/GW151012data.py) :  

In [7]:
for ifo in ifos :
                             ts[ifo] = strain[ifo].time_slice((tc - 75.), (tc + 75.))
                             ts3[ifo] = resstrain[ifo].time_slice((tc - 75.), (tc + 75.))

ts = res.whiten_wv(ts, f_low, f_high)
ts3 = res.whiten_wv(ts3, f_low, f_high)

creating /tmp/weave-kuzaibeir-TZh6sU/python27_intermediate/compiler_629fcf2110564006b7ec3a50ad9ba2cd


We may finally define the function that will estimate the correlation among the detectors in function of GPS time, given a certain time shift among the two detectors, we'll define that in a way that may be parallelized by using the _multiprocessing_ module :


In [8]:
def CorrAnal(i) :
      
              print ("Percentage of completition : ", ((df.Timeshift[i]*100)/0.2))              
              tau, corr = res.cross_correlation(ts['H1'], ts['L1'], (basetime + df.Timeshift[i])) 
              tau, corr_null = res.cross_correlation(ts3['H1'], ts3['L1'], (basetime + df.Timeshift[i]))
              df.SigCorr[i], df.ResCorr[i] = res.corr_wsgn_near_ml(corr,timedl,tdlerr), res.corr_wsgn_near_ml(corr_null,timedl,tdlerr)
              df.DiffCorr[i] = abs(df.SigCorr[i] - df.ResCorr[i])
              if(abs(df.ResCorr[i]) > abs(df.SigCorr[i])) : df.DiffCorr[i] = - df.DiffCorr[i]
              return [df.Timeshift[i], df.SigCorr[i], df.ResCorr[i], df.DiffCorr[i]]

The previous function, will be now run over <em>6</em> cores, the reason that lead us to choose <em>6</em> cores is that this pipeline was run in a laptop having <em>4</em> physical cores virtualized to <em>8</em>.
The number of cores used for the parallelization may still be decided in function of the machine used to run that, note though that a number too high in cores may result in slowing down due to processes queue :

In [9]:
if __name__ == '__main__':
         results = {}
         p = Pool(6)
         results = p.map(CorrAnal,rangenum)

('Percentage of completition : ', 0.0)
('Percentage of completition : ', 4.202101050525262)
('Percentage of completition : ', 12.606303151575787)
('Percentage of completition : ', 8.404202101050524)
('Percentage of completition : ', 16.80840420210105)
('Percentage of completition : ', 21.010505252626313)
('Percentage of completition : ', 21.060530265132567)
('Percentage of completition : ', 0.05002501250625313)
('Percentage of completition : ', 12.656328164082039)
('Percentage of completition : ', 8.454227113556778)
('Percentage of completition : ', 16.858429214607302)
('Percentage of completition : ', 4.252126063031516)
('Percentage of completition : ', 21.11055527763882)
('Percentage of completition : ', 0.10005002501250626)
('Percentage of completition : ', 16.908454227113555)
('Percentage of completition : ', 12.706353176588294)
('Percentage of completition : ', 8.504252126063031)
('Percentage of completition : ', 4.302151075537769)
('Percentage of completition : ', 21.160580290145

('Percentage of completition : ', 9.654827413706855)
('Percentage of completition : ', 5.502751375687844)
('Percentage of completition : ', 13.856928464232116)
('Percentage of completition : ', 1.3006503251625812)
('Percentage of completition : ', 18.10905452726363)
('Percentage of completition : ', 22.311155577788895)
('Percentage of completition : ', 9.704852426213106)
('Percentage of completition : ', 5.552776388194097)
('Percentage of completition : ', 13.90695347673837)
('Percentage of completition : ', 1.3506753376688343)
('Percentage of completition : ', 18.159079539769884)
('Percentage of completition : ', 9.754877438719358)
('Percentage of completition : ', 22.36118059029515)
('Percentage of completition : ', 5.60280140070035)
('Percentage of completition : ', 13.956978489244621)
('Percentage of completition : ', 18.209104552276138)
('Percentage of completition : ', 1.4007003501750874)
('Percentage of completition : ', 9.804902451225614)
('Percentage of completition : ', 22.41

('Percentage of completition : ', 15.107553776888444)
('Percentage of completition : ', 2.551275637818909)
('Percentage of completition : ', 23.561780890445227)
('Percentage of completition : ', 10.955477738869437)
('Percentage of completition : ', 19.409704852426213)
('Percentage of completition : ', 6.803401700850425)
('Percentage of completition : ', 15.157578789394698)
('Percentage of completition : ', 23.611805902951478)
('Percentage of completition : ', 2.6013006503251623)
('Percentage of completition : ', 11.005502751375689)
('Percentage of completition : ', 19.459729864932466)
('Percentage of completition : ', 6.853426713356678)
('Percentage of completition : ', 15.207603801900952)
('Percentage of completition : ', 23.661830915457724)
('Percentage of completition : ', 2.6513256628314155)
('Percentage of completition : ', 19.509754877438716)
('Percentage of completition : ', 11.05552776388194)
('Percentage of completition : ', 15.257628814407203)
('Percentage of completition : '

('Percentage of completition : ', 16.358179089544773)
('Percentage of completition : ', 20.710355177588795)
('Percentage of completition : ', 12.206103051525762)
('Percentage of completition : ', 3.851925962981491)
('Percentage of completition : ', 24.862431215607803)
('Percentage of completition : ', 8.104052026013006)
('Percentage of completition : ', 16.408204102051027)
('Percentage of completition : ', 20.76038019009505)
('Percentage of completition : ', 3.9019509754877437)
('Percentage of completition : ', 24.912456228114056)
('Percentage of completition : ', 12.256128064032016)
('Percentage of completition : ', 8.15407703851926)
('Percentage of completition : ', 16.458229114557277)
('Percentage of completition : ', 20.8104052026013)
('Percentage of completition : ', 3.9519759879939973)
('Percentage of completition : ', 24.962481240620306)
('Percentage of completition : ', 8.204102051025513)
('Percentage of completition : ', 12.306153076538271)
('Percentage of completition : ', 16

('Percentage of completition : ', 30.415207603801903)
('Percentage of completition : ', 38.71935967983992)
('Percentage of completition : ', 26.21310655327664)
('Percentage of completition : ', 47.073536768384194)
('Percentage of completition : ', 42.87143571785892)
('Percentage of completition : ', 34.56728364182091)
('Percentage of completition : ', 30.465232616308157)
('Percentage of completition : ', 38.76938469234618)
('Percentage of completition : ', 26.263131565782896)
('Percentage of completition : ', 42.92146073036518)
('Percentage of completition : ', 47.123561780890455)
('Percentage of completition : ', 34.61730865432717)
('Percentage of completition : ', 38.819409704852426)
('Percentage of completition : ', 30.515257628814407)
('Percentage of completition : ', 26.313156578289146)
('Percentage of completition : ', 42.97148574287144)
('Percentage of completition : ', 47.173586793396694)
('Percentage of completition : ', 34.667333666833414)
('Percentage of completition : ', 30

...

('Percentage of completition : ', 69.03451725862932)
('Percentage of completition : ', 56.52826413206603)
('Percentage of completition : ', 52.32616308154077)
('Percentage of completition : ', 73.23661830915458)
('Percentage of completition : ', 64.8824412206103)
('Percentage of completition : ', 69.08454227113556)
('Percentage of completition : ', 60.780390195097546)
('Percentage of completition : ', 56.57828914457228)
('Percentage of completition : ', 73.28664332166083)
('Percentage of completition : ', 52.37618809404702)
('Percentage of completition : ', 64.93246623311656)
('Percentage of completition : ', 69.13456728364181)
('Percentage of completition : ', 60.83041520760381)
('Percentage of completition : ', 73.33666833416709)
('Percentage of completition : ', 52.42621310655328)
('Percentage of completition : ', 56.62831415707854)
('Percentage of completition : ', 64.98249124562281)
('Percentage of completition : ', 69.18459229614808)
('Percentage of completition : ', 60.880440220

('Percentage of completition : ', 57.82891445722861)
('Percentage of completition : ', 74.48724362181092)
('Percentage of completition : ', 66.13306653326664)
('Percentage of completition : ', 62.03101550775388)
('Percentage of completition : ', 53.67683841920961)
('Percentage of completition : ', 57.878939469734874)
('Percentage of completition : ', 70.38519259629814)
('Percentage of completition : ', 74.53726863431716)
('Percentage of completition : ', 66.18309154577288)
('Percentage of completition : ', 62.08104052026013)
('Percentage of completition : ', 53.726863431715856)
('Percentage of completition : ', 57.92896448224111)
('Percentage of completition : ', 70.43521760880441)
('Percentage of completition : ', 66.23311655827914)
('Percentage of completition : ', 62.131065532766385)
('Percentage of completition : ', 74.58729364682341)
('Percentage of completition : ', 53.77688844422211)
('Percentage of completition : ', 57.978989494747374)
('Percentage of completition : ', 70.48524

('Percentage of completition : ', 80.1400700350175)
('Percentage of completition : ', 88.44422211105552)
('Percentage of completition : ', 96.74837418709355)
('Percentage of completition : ', 92.6463231615808)
('Percentage of completition : ', 75.9879939969985)
('Percentage of completition : ', 84.39219609804903)
('Percentage of completition : ', 80.19009504752377)
('Percentage of completition : ', 96.79839919959979)
('Percentage of completition : ', 88.49424712356178)
('Percentage of completition : ', 92.69634817408703)
('Percentage of completition : ', 76.03801900950475)
('Percentage of completition : ', 84.44222111055528)
('Percentage of completition : ', 80.24012006003001)
('Percentage of completition : ', 88.54427213606805)
('Percentage of completition : ', 96.84842421210605)
('Percentage of completition : ', 92.7463731865933)
('Percentage of completition : ', 76.088044022011)
('Percentage of completition : ', 84.49224612306153)
('Percentage of completition : ', 80.29014507253628)

('Percentage of completition : ', 85.64282141070535)
('Percentage of completition : ', 93.89694847423712)
('Percentage of completition : ', 98.04902451225612)
('Percentage of completition : ', 81.49074537268633)
('Percentage of completition : ', 77.28864432216108)
('Percentage of completition : ', 89.79489744872436)
('Percentage of completition : ', 85.6928464232116)
('Percentage of completition : ', 93.94697348674336)
('Percentage of completition : ', 98.09904952476239)
('Percentage of completition : ', 81.54077038519259)
('Percentage of completition : ', 77.33866933466733)
('Percentage of completition : ', 89.84492246123061)
('Percentage of completition : ', 85.74287143571784)
('Percentage of completition : ', 98.14907453726863)
('Percentage of completition : ', 93.99699849924961)
('Percentage of completition : ', 81.59079539769886)
('Percentage of completition : ', 77.38869434717358)
('Percentage of completition : ', 89.89494747373686)
('Percentage of completition : ', 85.7928964482

('Percentage of completition : ', 86.94347173586794)
('Percentage of completition : ', 95.1975987993997)
('Percentage of completition : ', 91.0455227613807)
('Percentage of completition : ', 99.34967483741872)
('Percentage of completition : ', 82.79139569784893)
('Percentage of completition : ', 78.58929464732367)
('Percentage of completition : ', 86.99349674837418)
('Percentage of completition : ', 95.24762381190595)
('Percentage of completition : ', 91.09554777388695)
('Percentage of completition : ', 99.39969984992497)
('Percentage of completition : ', 82.84142071035518)
('Percentage of completition : ', 78.63931965982992)
('Percentage of completition : ', 95.29764882441222)
('Percentage of completition : ', 91.1455727863932)
('Percentage of completition : ', 87.04352176088044)
('Percentage of completition : ', 82.89144572286142)
('Percentage of completition : ', 99.44972486243121)
('Percentage of completition : ', 78.68934467233616)
('Percentage of completition : ', 95.347673836918

As you may notice from the printing of the percentage of completition, the execution of the command wouldn't be linear, that's because when using _mpi_ the steps on which the instructions need to be run, passed to the function through the _rangenum_ array, will be sliced and given to the various core.
To track down the execution thoughh you may just check the average value of percentage !
We now need to reorder the created Dataframe in order to save that : 

In [10]:
for i in rangenum:
         for j in rangenum:
             if(df.Timeshift[i] == results[j][0]):
                 df.SigCorr[i] = results[j][1]
                 df.ResCorr[i] = results[j][2]
                 df.DiffCorr[i] = results[j][3]
                 break

Lastly, we just need to save the created dataframe in a _.csv_ file :

In [11]:
df.to_csv('CorrVsTime.csv',index = False)

Your [CorrVsTime.csv](https://github.com/GravWaves-IMF/Correlation-Method-first-2019-/blob/master/Code/GW151012Final/GW151012data.py) dataframe is finally ready to be analyzed ! 